## Compare likelihoods for theories with different values of H_0

We redo the notebook full_likelihood_example, but now varying also H_0

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 140
mpl.rcParams['figure.dpi'] = 140
# our modules
import data_MPGADGET
import p1d_arxiv
import gp_emulator
import likelihood
import likelihood_parameter
import lya_theory
import full_theory
import camb_cosmo
import CAMB_model
import fit_linP

### Setup mock data, from one of simulations in the suite

In [2]:
# specify simulation suite and P1D mesurements
basedir="/p1d_emulator/sim_suites/Australia20/"
skewers_label='Ns500_wM0.05'
p1d_label=None
# specify simulation to use to generate synthetic data
test_sim_num=15
data=data_MPGADGET.P1D_MPGADGET(sim_number=test_sim_num,basedir=basedir,skewers_label=skewers_label,data_cov_factor=1)

### Setup archive of P1D measurements from simulations and train the emulator

In [3]:
# specify redshift range
z_max=4
# do not use test_sim_num that was used in generating mock data
arxiv=p1d_arxiv.ArxivP1D(basedir=basedir,drop_sim_number=test_sim_num,
                            drop_tau_rescalings=True,z_max=z_max,
                            drop_temp_rescalings=True,skewers_label=skewers_label)

In [4]:
# specify parameters to be used in emulator
paramList=['mF', 'sigT_Mpc', 'gamma', 'kF_Mpc', 'Delta2_p', 'n_p']
# specify k range
kmax_Mpc=8
# setup GP emulator
emu=gp_emulator.GPEmulator(basedir,p1d_label,skewers_label,z_max=z_max,
                                verbose=False,paramList=paramList,train=True,
                                emu_type="k_bin",passArxiv=arxiv,kmax_Mpc=kmax_Mpc,
                                drop_tau_rescalings=True,
                                drop_temp_rescalings=True)

 /Users/font/Programs/Others/anaconda3/lib/python3.6/site-packages/paramz/parameterized.py:61: RuntimeWarning:Don't forget to initialize by self.initialize_parameter()!


Training GP on 270 points
Optimised


### Setup and compare theory objects

Start by setting up an old theory object, that uses only compressed linP parameters

In [5]:
old_theory=lya_theory.LyaTheory(zs=data.z,emulator=emu,verbose=True)
old_emu_calls=old_theory.get_emulator_calls()

0 2.0 model {'Delta2_p': 0.6298978537031776, 'n_p': -2.2993906458284794, 'alpha_p': -0.21641970780014216, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
1 2.25 model {'Delta2_p': 0.5399249525510917, 'n_p': -2.2993815689130663, 'alpha_p': -0.21642424974102223, 'f_p': 0.9666989020097663, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.13051263566195997, 'kF_Mpc': 9.574544756852742}
2 2.5 model {'Delta2_p': 0.4676198117634747, 'n_p': -2.2993726176961222, 'alpha_p': -0.21642898800315924, 'f_p': 0.9729433890617769, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12986720978610813, 'kF_Mpc': 9.91377946463774}
3 2.75 model {'Delta2_p': 0.4087278727714764, 'n_p': -2.2993635887568735, 'alpha_p': -0.2164333773453141, 'f_p': 0.9776963475296402, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.1290634736772292, 'kF_Mpc': 10.258134266642541}
4 3.

Setup now a new theory object, that holds a CAMB object

In [6]:
new_theory=full_theory.FullTheory(zs=data.z,emulator=emu,verbose=True)

In [7]:
# compare emulator calls at one of the redshifts
old_emu_calls=old_theory.get_emulator_calls()
new_emu_calls=new_theory.get_emulator_calls(return_M_of_z=False)

0 2.0 model {'Delta2_p': 0.6298978537031776, 'n_p': -2.2993906458284794, 'alpha_p': -0.21641970780014216, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
1 2.25 model {'Delta2_p': 0.5399249525510917, 'n_p': -2.2993815689130663, 'alpha_p': -0.21642424974102223, 'f_p': 0.9666989020097663, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.13051263566195997, 'kF_Mpc': 9.574544756852742}
2 2.5 model {'Delta2_p': 0.4676198117634747, 'n_p': -2.2993726176961222, 'alpha_p': -0.21642898800315924, 'f_p': 0.9729433890617769, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12986720978610813, 'kF_Mpc': 9.91377946463774}
3 2.75 model {'Delta2_p': 0.4087278727714764, 'n_p': -2.2993635887568735, 'alpha_p': -0.2164333773453141, 'f_p': 0.9776963475296402, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.1290634736772292, 'kF_Mpc': 10.258134266642541}
4 3.

In [8]:
print("z=2 emulator calls for old theory = ", old_emu_calls[0])
print("z=2 emulator calls for new theory = ", new_emu_calls[0])

z=2 emulator calls for old theory =  {'Delta2_p': 0.6298978537031776, 'n_p': -2.2993906458284794, 'alpha_p': -0.21641970780014216, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
z=2 emulator calls for new theory =  {'Delta2_p': 0.6301020569149606, 'n_p': -2.2994471404944634, 'alpha_p': -0.21652936468729186, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}


Note that the numbers above are not supposed to exactly match, since kp_Mpc is different in both theories

### Compare now emulator calls for a different test cosmology

In [9]:
As=1.9e-9
ns=0.92
H0=50.0
test_cosmo=camb_cosmo.get_cosmology(H0=H0,As=As,ns=ns)
test_camb_model=CAMB_model.CAMBModel(zs=data.z,cosmo=test_cosmo)

In [10]:
# print compressed linP parameters from the fiducial cosmology in the old theory
old_linP=fit_linP.LinearPowerModel(cosmo=old_theory.cosmo.cosmo_fid)
old_like_params=old_linP.get_likelihood_parameters()
for par in old_like_params:
    print(par.info_str())

g_star = 0.9677508579459803
f_star = 0.981369557840318
Delta2_star = 0.36017590528641974
n_star = -2.299335667257498
alpha_star = -0.2165270371206895


In [11]:
# print compressed linP parameters from the test cosmology
test_linP=fit_linP.LinearPowerModel(cosmo=test_cosmo)
old_test_like_params=test_linP.get_likelihood_parameters()
for par in old_test_like_params:
    print(par.info_str())

g_star = 0.9886588719453293
f_star = 0.9928886747421347
Delta2_star = 0.2898177280857949
n_star = -2.3419938318462292
alpha_star = -0.217671224372772


In [12]:
# we want to study also cases where we ignore some of the parameters
#cull_params=[]
cull_params=['f_star','g_star','alpha_star']
culled_like_params = [par for par in old_test_like_params if par.name not in cull_params]
for par in culled_like_params:
    print(par.info_str())

Delta2_star = 0.2898177280857949
n_star = -2.3419938318462292


In [13]:
# call the old theory, but using the freshly computed likelihood parameters from the test cosmology
old_test_emu_calls=old_theory.get_emulator_calls(like_params=old_test_like_params)

0 2.0 model {'Delta2_p': 0.5102215380511176, 'n_p': -2.3420488104172104, 'alpha_p': -0.21756389505221058, 'f_p': 0.9698760305702522, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.1319714402127672, 'kF_Mpc': 9.151699369590883}
1 2.25 model {'Delta2_p': 0.4365371414983146, 'n_p': -2.342039733501799, 'alpha_p': -0.21756843699310158, 'f_p': 0.978218018911583, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.131365305932712, 'kF_Mpc': 9.512397985205633}
2 2.5 model {'Delta2_p': 0.3774324411424346, 'n_p': -2.3420307822848527, 'alpha_p': -0.21757317525525846, 'f_p': 0.9844625059635936, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.1304122099326903, 'kF_Mpc': 9.87234920849685}
3 2.75 model {'Delta2_p': 0.32937473146887536, 'n_p': -2.3420217533456054, 'alpha_p': -0.21757756459740804, 'f_p': 0.9892154644314569, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.1293249702452728, 'kF_Mpc': 10.23739220190307}
4 3.0 mod

In [14]:
# same, but using only the culled parameters
old_test_emu_calls=old_theory.get_emulator_calls(like_params=culled_like_params)

0 2.0 model {'Delta2_p': 0.5068511308139868, 'n_p': -2.342048810417209, 'alpha_p': -0.21641970780014766, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
1 2.25 model {'Delta2_p': 0.43445388985904443, 'n_p': -2.342039733501798, 'alpha_p': -0.21642424974102525, 'f_p': 0.9666989020097663, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.13051263566195997, 'kF_Mpc': 9.574544756852742}
2 2.5 model {'Delta2_p': 0.37627311950649595, 'n_p': -2.3420307822848527, 'alpha_p': -0.21642898800316954, 'f_p': 0.9729433890617769, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12986720978610813, 'kF_Mpc': 9.91377946463774}
3 2.75 model {'Delta2_p': 0.3288853633831223, 'n_p': -2.342021753345605, 'alpha_p': -0.21643337734533125, 'f_p': 0.9776963475296402, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.1290634736772292, 'kF_Mpc': 10.258134266642541}
4 3.

In [15]:
# print cosmology parameters from the new theory for the fiducial cosmology
new_like_params=new_theory.camb_model_fid.get_likelihood_parameters()
for par in new_like_params:
    print(par.info_str())

ombh2 = 0.022
omch2 = 0.12
As = 2.1e-09
ns = 0.965
H0 = 67.0


In [16]:
# print cosmology parameters from new test camb model
new_test_like_params=test_camb_model.get_likelihood_parameters()
for par in new_test_like_params:
    print(par.info_str())

ombh2 = 0.022
omch2 = 0.12
As = 1.9e-09
ns = 0.92
H0 = 50.0


In [17]:
# call the new theory, but using the freshly computed likelihood parameters for the test cosmology
new_test_emu_calls=new_theory.get_emulator_calls(like_params=new_test_like_params,return_M_of_z=False)

0 2.0 model {'Delta2_p': 0.5158301022028047, 'n_p': -2.3444512944977385, 'alpha_p': -0.21653333140739928, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.134042289894799, 'kF_Mpc': 9.010312694203272}
1 2.25 model {'Delta2_p': 0.4405094871015889, 'n_p': -2.3444513840392465, 'alpha_p': -0.21653343114322543, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.13308974073523247, 'kF_Mpc': 9.389146485499525}
2 2.5 model {'Delta2_p': 0.3804645956308694, 'n_p': -2.3444513656889594, 'alpha_p': -0.21653329025556498, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.1319330557436291, 'kF_Mpc': 9.758546637539629}
3 2.75 model {'Delta2_p': 0.3318537959707248, 'n_p': -2.3444514195666235, 'alpha_p': -0.21653305386771934, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.13074044033566992, 'kF_Mpc': 10.126556392965496}
4 3.0 model {'Delta2_p': 0.2919631977506819, 'n_p': -2.344451310866517, 'alpha_p': -0.21653370642470238, 'mF': 0

In [18]:
print("z=2 emulator calls for old test theory = ", old_test_emu_calls[0])
print("z=2 emulator calls for new test theory = ", new_test_emu_calls[0])

z=2 emulator calls for old test theory =  {'Delta2_p': 0.5068511308139868, 'n_p': -2.342048810417209, 'alpha_p': -0.21641970780014766, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
z=2 emulator calls for new test theory =  {'Delta2_p': 0.5158301022028047, 'n_p': -2.3444512944977385, 'alpha_p': -0.21653333140739928, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.134042289894799, 'kF_Mpc': 9.010312694203272}


In [19]:
print("z = {} emulator calls for old test theory = {}".format(data.z[-1],old_test_emu_calls[-1]))
print("z = {} emulator calls for new test theory = {}".format(data.z[-1],new_test_emu_calls[-1]))

z = 4.0 emulator calls for old test theory = {'Delta2_p': 0.18663018539044188, 'n_p': -2.3419795937653722, 'alpha_p': -0.21645453488671731, 'f_p': 0.9898232760729386, 'mF': 0.3667549106286143, 'gamma': 1.4366944565860078, 'sigT_Mpc': 0.1124868806234996, 'kF_Mpc': 12.841201989686333}
z = 4.0 emulator calls for new test theory = {'Delta2_p': 0.18732075320994157, 'n_p': -2.344451269344503, 'alpha_p': -0.21653362510852903, 'mF': 0.3667549106286143, 'gamma': 1.4366944565860078, 'sigT_Mpc': 0.11311066853561205, 'kF_Mpc': 12.770384738918878}


Both new and old theories make the same emulator call for a theory that is not the fiducial!

### Test the likelihood objects with old and new theories

In [20]:
## Set up a likelihood object with the old theory object
#old_free_params=['Delta2_star','n_star']
old_free_params=['Delta2_star','n_star','alpha_star','f_star','g_star']
old_like=likelihood.Likelihood(data=data,theory=old_theory,
                            free_parameters=old_free_params,
                            free_param_limits=None,
                            verbose=True,
                            prior_Gauss_rms=0.15,
                            emu_cov_factor=1)

got parameters
g_star = 0.9677508579459803
f_star = 0.981369557840318
Delta2_star = 0.36017590528641974
n_star = -2.299335667257498
alpha_star = -0.2165270371206895
ln_tau_0 = 0.0
ln_tau_1 = 0.0
ln_sigT_kms_0 = 0.0
ln_sigT_kms_1 = 0.0
ln_gamma_0 = 0.0
ln_gamma_1 = 0.0
ln_kF_0 = 0.0
ln_kF_1 = 0.0
likelihood setup with 5 free parameters
5 free parameters


In [21]:
# get parameter values in cube for test cosmology
old_test_values=old_like.sampling_point_from_parameters(old_test_like_params)
# check that parameters inferred from these are same than computed before
for par in old_like.parameters_from_sampling_point(values=old_test_values):
    print(par.info_str(),',',par.value_in_cube())

g_star = 0.9886588719453293 , 0.966471798633233
f_star = 0.9928886747421347 , 1.072216868553368
Delta2_star = 0.2898177280857949 , 0.26545152057196614
n_star = -2.3419938318462292 , 0.08006168153770858
alpha_star = -0.217671224372772 , 0.47571614206570917


In [22]:
old_like.get_log_like(values=old_test_values,ignore_log_det_cov=False)

0 2.0 model {'Delta2_p': 0.5102215380511176, 'n_p': -2.3420488104172104, 'alpha_p': -0.21756389505221058, 'f_p': 0.9698760305702522, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.1319714402127672, 'kF_Mpc': 9.151699369590883}
1 2.25 model {'Delta2_p': 0.4365371414983146, 'n_p': -2.342039733501799, 'alpha_p': -0.21756843699310158, 'f_p': 0.978218018911583, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.131365305932712, 'kF_Mpc': 9.512397985205633}
2 2.5 model {'Delta2_p': 0.3774324411424346, 'n_p': -2.3420307822848527, 'alpha_p': -0.21757317525525846, 'f_p': 0.9844625059635936, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.1304122099326903, 'kF_Mpc': 9.87234920849685}
3 2.75 model {'Delta2_p': 0.32937473146887536, 'n_p': -2.3420217533456054, 'alpha_p': -0.21757756459740804, 'f_p': 0.9892154644314569, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.1293249702452728, 'kF_Mpc': 10.23739220190307}
4 3.0 mod

34.88183733050968

In [23]:
## Set up a likelihood object with the old theory object, but now with only 2 key parameters
old_free_params=['Delta2_star','n_star']
old_like=likelihood.Likelihood(data=data,theory=old_theory,
                            free_parameters=old_free_params,
                            free_param_limits=None,
                            verbose=True,
                            prior_Gauss_rms=0.15,
                            emu_cov_factor=1)

got parameters
g_star = 0.9677508579459803
f_star = 0.981369557840318
Delta2_star = 0.36017590528641974
n_star = -2.299335667257498
alpha_star = -0.2165270371206895
ln_tau_0 = 0.0
ln_tau_1 = 0.0
ln_sigT_kms_0 = 0.0
ln_sigT_kms_1 = 0.0
ln_gamma_0 = 0.0
ln_gamma_1 = 0.0
ln_kF_0 = 0.0
ln_kF_1 = 0.0
likelihood setup with 2 free parameters
2 free parameters


In [24]:
# get parameter values in cube for test cosmology
old_test_values=old_like.sampling_point_from_parameters(old_test_like_params)
# check that parameters inferred from these are same than computed before
for par in old_like.parameters_from_sampling_point(values=old_test_values):
    print(par.info_str(),',',par.value_in_cube())

Delta2_star = 0.2898177280857949 , 0.26545152057196614
n_star = -2.3419938318462292 , 0.08006168153770858


In [25]:
old_like.get_log_like(values=old_test_values,ignore_log_det_cov=False)

0 2.0 model {'Delta2_p': 0.5068511308139868, 'n_p': -2.342048810417209, 'alpha_p': -0.21641970780014766, 'f_p': 0.9583569136684355, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.13078611068417573, 'kF_Mpc': 9.234642271117814}
1 2.25 model {'Delta2_p': 0.43445388985904443, 'n_p': -2.342039733501798, 'alpha_p': -0.21642424974102525, 'f_p': 0.9666989020097663, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.13051263566195997, 'kF_Mpc': 9.574544756852742}
2 2.5 model {'Delta2_p': 0.37627311950649595, 'n_p': -2.3420307822848527, 'alpha_p': -0.21642898800316954, 'f_p': 0.9729433890617769, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.12986720978610813, 'kF_Mpc': 9.91377946463774}
3 2.75 model {'Delta2_p': 0.3288853633831223, 'n_p': -2.342021753345605, 'alpha_p': -0.21643337734533125, 'f_p': 0.9776963475296402, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.1290634736772292, 'kF_Mpc': 10.258134266642541}
4 3.

34.554831342096485

In [26]:
## Set up a likelihood object with the new theory object
new_free_params=["As","ns","H0"]
new_like=likelihood.Likelihood(data=data,theory=new_theory,
                            free_parameters=new_free_params,
                            free_param_limits=None,
                            verbose=True,
                            prior_Gauss_rms=0.15,
                            emu_cov_factor=1)

got parameters
ombh2 = 0.022
omch2 = 0.12
As = 2.1e-09
ns = 0.965
H0 = 67.0
ln_tau_0 = 0.0
ln_tau_1 = 0.0
ln_sigT_kms_0 = 0.0
ln_sigT_kms_1 = 0.0
ln_gamma_0 = 0.0
ln_gamma_1 = 0.0
ln_kF_0 = 0.0
ln_kF_1 = 0.0
likelihood setup with 3 free parameters
3 free parameters


In [27]:
# get parameter values in cube for test cosmology in new theory
new_test_values=new_like.sampling_point_from_parameters(new_test_like_params)
# check that parameters inferred from these are same than computed before
for par in new_like.parameters_from_sampling_point(values=new_test_values):
    print(par.info_str(),',',par.value_in_cube())

As = 1.9e-09 , -0.500000000000001
ns = 0.92 , -0.9999999999999963
H0 = 50.0 , -0.9285714285714286


In [28]:
new_like.get_log_like(values=new_test_values,ignore_log_det_cov=False)

0 2.0 model {'Delta2_p': 0.5158301022028047, 'n_p': -2.3444512944977385, 'alpha_p': -0.21653333140739928, 'mF': 0.8507256483811173, 'gamma': 1.5534927645653829, 'sigT_Mpc': 0.134042289894799, 'kF_Mpc': 9.010312694203272}
1 2.25 model {'Delta2_p': 0.4405094871015889, 'n_p': -2.3444513840392465, 'alpha_p': -0.21653343114322543, 'mF': 0.8145423945568311, 'gamma': 1.5449855838581277, 'sigT_Mpc': 0.13308974073523247, 'kF_Mpc': 9.389146485499525}
2 2.5 model {'Delta2_p': 0.3804645956308694, 'n_p': -2.3444513656889594, 'alpha_p': -0.21653329025556498, 'mF': 0.7709160253724354, 'gamma': 1.535466821877, 'sigT_Mpc': 0.1319330557436291, 'kF_Mpc': 9.758546637539629}
3 2.75 model {'Delta2_p': 0.3318537959707248, 'n_p': -2.3444514195666235, 'alpha_p': -0.21653305386771934, 'mF': 0.7191184346490203, 'gamma': 1.5246254858011445, 'sigT_Mpc': 0.13074044033566992, 'kF_Mpc': 10.126556392965496}
4 3.0 model {'Delta2_p': 0.2919631977506819, 'n_p': -2.344451310866517, 'alpha_p': -0.21653370642470238, 'mF': 0

36.93504215563791